In [31]:
# import important library
import pandas as pd
import numpy as np
import bot

In [61]:
import importlib
importlib.reload(bot)

<module 'bot' from '/Users/henrytran/Documents/GitHub/aias_trading25/bot.py'>

# 1. Data Processing

In [2]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [ ]:
# convert time strings into timestamps, and take only year, month and date values
daily_df["date"]=pd.to_datetime(daily_df["date"]).dt.strftime("%Y-%m-%d")

In [4]:
daily_df.shape

(2651, 9)

The dataset has 2651 records from 2014 to 2022.

As the data is needed for training and testing, every records before 2020 will be used for training and every records after 2020 will be used for testing.

In [5]:
# splitting the data
training = daily_df[daily_df["date"]<"2020-01-01"]
testing = daily_df[daily_df["date"]>="2020-01-01"]

In [66]:
# convert the data into csv files
training.to_csv('training.csv')
testing.to_csv('testing.csv')

After splitting, the training dataset has 1860 records and the testing data has 791 records.

In [6]:
print(training.shape)
print(testing.shape)

(1860, 9)
(791, 9)


# 2. Hill-climbing algorithm

## a. Try the bot

In [7]:
# extract the data
close_price_train=training["close"]
no_of_day_train=training.shape[0]
close_price_test =testing["close"]
no_of_day_test=testing.shape[0]

In [8]:
low_sma = bot.wma(close_price_train, 10, bot.sma_filter(10))
high_sma = bot.wma(close_price_train, 20, bot.sma_filter(20))

print(low_sma)
print(len(low_sma))
print(high_sma)
print(len(high_sma))

[-5844.114 -4371.12  -2900.949 ...   371.484   374.107   375.274]
1860
[-6466.681  -5746.248  -5014.0975 ...   354.9755   357.9115   361.117 ]
1860


In [63]:
buy_sell_signals=[]
for i in range(len(low_sma)-1):
    if low_sma[i]<high_sma[i] and low_sma[i+1]>high_sma[i+1]:
        buy_sell_signals.append("buy")
    elif low_sma[i]>high_sma[i] and low_sma[i+1]<high_sma[i+1]:
        buy_sell_signals.append("sell")
    else:
        buy_sell_signals.append("-")

In [10]:
def bot_sma(close_price, hfw, lfw):
    low_sma = bot.wma(close_price, lfw, bot.sma_filter(lfw))
    high_sma = bot.wma(close_price, hfw, bot.sma_filter(hfw))
    buy_sell_signals=[]
    for i in range(len(low_sma)-1):
        if low_sma[i]<high_sma[i] and low_sma[i+1]>high_sma[i+1]:
            buy_sell_signals.append("buy")
        elif low_sma[i]>high_sma[i] and low_sma[i+1]<high_sma[i+1]:
            buy_sell_signals.append("sell")
        else:
            buy_sell_signals.append("-")
    return buy_sell_signals

In [12]:
np.random.randint(-5,5)

3

## b. Hill-climbing algorithm

In [68]:
# Pseudocode for hill-climbing 

def hill_climbing(close_price, high_frequency_window, low_frequency_window, max_iter=100):
    if 11 <= high_frequency_window <= 40 and 2 <= low_frequency_window <= 10:
        # run the first bot
        high_window = high_frequency_window
        low_window = low_frequency_window
        for i in range(max_iter):
            # run the second bot with tweak parameter
            new_high_frequency_window, new_low_frequency_window = parameter_tweak(high_frequency_window, low_frequency_window)
            cash2=bot_fitness_func(new_high_frequency_window, new_low_frequency_window)
            cash1=bot_fitness_func(high_window, low_window) 
            if  cash2> cash1:
                high_window = new_high_frequency_window
                low_window = new_low_frequency_window
                bot1=bot2
                cash1=cash2
        return high_frequency_window, low_frequency_window, cash1

def parameter_tweak(hfw, lfw):
    rng = np.random.default_rng()
    a=rng.integers(-5,6)
    new_hfw=hfw+a
    b=rng.integers(-5,6)
    new_lfw=lfw+b
    # we check to make sure that new_hfw in range(11,40) and new_lfw(2,10)
    new_hfw = max(11, min(40, new_hfw))
    new_lfw = max(2, min(10, new_lfw))
    if new_hfw > new_lfw:
        return new_hfw, new_lfw
    else:
        return parameter_tweak(new_hfw, new_lfw)


In [69]:
hfw_result, lfw_result, cash_result = hill_climbing(close_price_train, 20,10)
hfw_result, lfw_result, cash_result

(20, 10, 1000)

In [47]:
# final cash amount
cash_result

1000

# 3. Fitness function and Bot evaluation function

In [70]:
def bot_fitness_func(high_window, low_window): # change bot_signals to high_frequency_window_size, low_frequency_window_size
    """
    This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time: recorded trading time (daily)
    Return:
    - cash: the total cash amount earned after the trading time
    """ 
    
    # call the bot in here
    # Jackie looks after this part - just need to call the bot with 2 parameters (high_frequency_window_size, low_frequency_window_size)
    
    # intialise bot, use training dataset for optimisation algorithms fitness functions
    close_price = pd.read_csv('training.csv')['close']
    bot_signals = bot.get_signals_sma2(close_price, high_window, low_window) #need to think about how to call 2 other bot algorithms

    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(len(close_price)-1):
        close=close_price.iloc[i]
        # buy, ensure we have cash to buy
        if bot_signals[i] == "buy" and cash>0:
            bitcoin =  (cash*(1-fee))/close
            cash = 0
        # sell, ensure we have bitcoin to sell
        elif bot_signals[i] == "sell" and bitcoin>0:
            cash = bitcoin * close * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    last_close=close_price.iloc[-1]
    if bitcoin>0:
        cash = bitcoin * last_close * (1-fee)
        bitcoin =0
    
    return cash

In [ ]:
def bot_evaluation(bot_signals, time, close_price):
    """
    This function will calculate the total cash earned from the buy/sell signals of the trading bot.
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time: recorded trading time (daily)
    Return:
    - result: a nested list, with each smallest contains the time of the transaction, the amount of cash and amount of bitcoin
    """ 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    result =[]
    
    #loop through the time length
    for i in range(len(time)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
        result.append([time[i],cash, bitcoin])
    
    return result

In [ ]:
#def print_nicely(list):
    